In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
## GBIF TAXONOMIC BACKBONE
backbone_original = pd.read_csv("../../data/external/backbone/Taxon.tsv", sep="\t", on_bad_lines='skip')

C:\Users\melanie\AppData\Local\Temp\ipykernel_10612\1960394640.py:2: DtypeWarning: Columns (9,10,16,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  backbone_original = pd.read_csv("../../data/external/backbone/Taxon.tsv", sep="\t", on_bad_lines='skip')


In [7]:
backbone_original.loc[100010]

taxonID                                                  7635514
datasetID                   2d59e5db-57ad-41ff-97d6-11f5fb264527
parentNameUsageID                                      2113911.0
acceptedNameUsageID                                    6467897.0
originalNameUsageID                                          NaN
scientificName                      Scalpellum dubium Hoek, 1883
scientificNameAuthorship                              Hoek, 1883
canonicalName                                  Scalpellum dubium
genericName                                           Scalpellum
specificEpithet                                           dubium
infraspecificEpithet                                         NaN
taxonRank                                                species
nameAccordingTo                                              NaN
namePublishedIn                                              NaN
taxonomicStatus                                          synonym
nomenclaturalStatus      

In [12]:
backbone_original[backbone_original["canonicalName"]=='Palmaepollenites kutchensis']

,taxonID,datasetID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificName,scientificNameAuthorship,canonicalName,genericName,specificEpithet,...,namePublishedIn,taxonomicStatus,nomenclaturalStatus,taxonRemarks,kingdom,phylum,class,order,family,genus
1238642,11828001,c33ce2f2-c3cc-43a5-a380-fe4526d63650,4909081.0,NaN,NaN,Palmaepollenites kutchensis Venkatachala & Kar...,"Venkatachala & Kar, 1969",Palmaepollenites kutchensis,Palmaepollenites,kutchensis,...,NaN,accepted,NaN,NaN,Plantae,Tracheophyta,Liliopsida,Arecales,Arecaceae,Palmaepollenites


In [14]:
backbone_original.loc[1238642]

taxonID                                                              11828001
datasetID                                c33ce2f2-c3cc-43a5-a380-fe4526d63650
parentNameUsageID                                                   4909081.0
acceptedNameUsageID                                                       NaN
originalNameUsageID                                                       NaN
scientificName              Palmaepollenites kutchensis Venkatachala & Kar...
scientificNameAuthorship                             Venkatachala & Kar, 1969
canonicalName                                     Palmaepollenites kutchensis
genericName                                                  Palmaepollenites
specificEpithet                                                    kutchensis
infraspecificEpithet                                                      NaN
taxonRank                                                             species
nameAccordingTo                                                 

In [15]:
backbone = backbone_original.dropna(subset="canonicalName").set_index("canonicalName")
len(backbone)

6420169

In [17]:
backbone.loc['Palmaepollenites kutchensis']

taxonID                                                              11828001
datasetID                                c33ce2f2-c3cc-43a5-a380-fe4526d63650
parentNameUsageID                                                   4909081.0
acceptedNameUsageID                                                       NaN
originalNameUsageID                                                       NaN
scientificName              Palmaepollenites kutchensis Venkatachala & Kar...
scientificNameAuthorship                             Venkatachala & Kar, 1969
genericName                                                  Palmaepollenites
specificEpithet                                                    kutchensis
infraspecificEpithet                                                      NaN
taxonRank                                                             species
nameAccordingTo                                                           NaN
namePublishedIn                                                 

In [16]:
len(backbone_original)

7612133

In [17]:
# PREPROCESSING
## AUTHORS
authors = pd.read_pickle("../../data/interim/european_taxonomic_authors_no_duplicates.pkl")
# less columns and author ID as index quicken processing
authors = authors[["author_id", "author_display_name", "orcid",
                   "inst_id", "inst_display_name",  "species_subject"]]
authors = authors.set_index("author_id")

In [18]:
# get initial first name + last name for every author
truncated_names = []

for author in authors.itertuples():
    first_initial = author.author_display_name[0]
    last_name = author.author_display_name.split(" ")[-1]
    truncated_names.append(first_initial + " " + last_name)
    
authors["truncatedName"] = truncated_names

In [34]:
backbone = backbone_original[backbone_original["taxonomicStatus"]!="doubtful"]
backbone = backbone[["canonicalName", "order"]]
backbone = backbone.dropna().drop_duplicates(ignore_index=True).reset_index(drop=True)
len(backbone.index)

5361501

In [73]:
len(set(backbone_original[backbone_original["kingdom"]=="Plantae"]["family"]))

1628

In [37]:
backbone = backbone_original[backbone_original["taxonomicStatus"]!="doubtful"]
backbone = backbone[["canonicalName", "kingdom", "phylum","order"]]
backbone = backbone.dropna().drop_duplicates(ignore_index=True).reset_index(drop=True)
len(backbone.index)

5359741

In [35]:
backbone = backbone_original[backbone_original["taxonomicStatus"]!="doubtful"]
backbone = backbone[["canonicalName", "family"]]
backbone = backbone.dropna().drop_duplicates(ignore_index=True).reset_index(drop=True)
len(backbone.index)

5551026

In [36]:
backbone = backbone_original[backbone_original["taxonomicStatus"]!="doubtful"]
backbone = backbone[["canonicalName", "kingdom"]]
backbone = backbone.dropna().drop_duplicates(ignore_index=True).reset_index(drop=True)
len(backbone.index)

5596652

In [76]:
backbone = backbone_original[backbone_original["taxonomicStatus"]!="doubtful"]
backbone = backbone[["canonicalName", "order", "family"]]
backbone = backbone.dropna(subset="canonicalName").drop_duplicates(ignore_index=True).reset_index(drop=True)
len(backbone.index)

5626269

C:\Users\melanie\AppData\Local\Temp\ipykernel_21636\1960394640.py:2: DtypeWarning: Columns (9,10,16,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  backbone_original = pd.read_csv("../../data/external/backbone/Taxon.tsv", sep="\t", on_bad_lines='skip')


In [22]:
# reduce size of backbone for easier searching
backbone = backbone_original[backbone_original["taxonomicStatus"]!="doubtful"]
backbone = backbone[["canonicalName", "order"]]
# remove taxa with no known species name or order
backbone = backbone.dropna().drop_duplicates(ignore_index=True).reset_index(drop=True)

In [23]:
# backbone to dictionary for quicker processing
seen_species = {}

for species in backbone.itertuples():
    if species.canonicalName not in seen_species:
        seen_species[species.canonicalName] = species.order

In [25]:
## LINK AUTHORS TO BACKBONE
# start with empty list for order
authors["order"] = [list() for x in range(len(authors.index))]
    
# for every author, get order for every species they study
for i, author in authors.iterrows():
    for species in author["species_subject"]:
        if species in seen_species:
            # get order name according to GBIF
            order_name = seen_species[species]
            # add this order to the list of orders studied by the author (no duplicates)
            if order_name not in author["order"]:
                authors.loc[i, "order"].append(order_name)

Proxapertites operculatus
Rigonia plicata
Scutellastra mexicana
Cnemaspis omari
Cnemaspis kendallii
Cnemaspis temiah
Cnemaspis stongensis
Cnemaspis hangus
Cnemaspis sundagekko
Cnemaspis peninsularis
Cnemaspis mumpuniae
Cnemaspis sundainsula
Teira dugesii
Colymbosaurus megadeirus
Paludititan nalatzensis
Magyarosaurus dacus
Palaeacanthaspis vasta
Corumbella werneri
Hansenocaris itoi
Desbruyeresia melanioides
Desbruyeresia cancellata
Desbruyeresia costata
Carlia johnstonei
Carlia insularis
Carlia isostriacantha
Carlia triacantha
Pancytospora epiphaga
Myrmicinosporidium durum
Pelourdea vogesiaca
Saccoglossus kowalevskii
Schizocardium californicum
Dailyatia odyssei
Eohadrotreta zhenbaensis
Doratodon carcharidens
Abarenicola assimilis
Tarentola ephippiata
Tarentola annularis
Turcosuchus okani
Poecilochaetus brenkei
Laonice shulseae
Aurospio dibranchiata
Prionospio amarsupiata
Apicystis bombi
Melanopsis anistratenkoi
Melanopsis gearyae
Melanopsis magyari
Melanopsis vrcinensis
Melanopsis costa

Xenodens calminechari
Podarcis peloponnesiacus
Pararhabdodon isonensis
Gothograptus nassa
Gothograptus obtectus
Gothograptus domeyki
Gothograptus velo
Gothograptus diminutus
Gothograptus auriculatus
Radix balthica
Lymnaea stagnalis
Batropetes palatinus
Batropetes appelensis
Batropetes niederkirchensis
Varanus salvator
Saurichthys madagascariensis
Anguis fragilis
Anguis colchica
Eryops megacephalus
Acheloma dunni
Trimerorhachis insignis
Hierophis andreanus
Acosmia maotiania
Thelodus parvidens
Thelodus trilobatus
Paralogania ludlowiensis
Nostolepis gracilis
Desbruyeresia melanioides
Desbruyeresia cancellata
Desbruyeresia costata
Carcinus maenas
Scleromochlus taylori
Anolis sagrei
Europasaurus holgeri
Dermochelys coriacea
Amiskwia sagittiformis
Queiroziella brasiliensis
Trachemys scripta
Pseudoarthrographis phlogis
Melanoides tuberculata
Douvilleiceras mammillatum
Sternaspis scutata
Wasatchites distractus
Pristurus minimus
Scolecolachnum nigricans
Fuscolachnum pteridis
Natrix maura
Galba 

Saloumia gorodiskii
Acroloxus lacustris
Melanoides tuberculata
Armiger crista
Nothosaurus marchicus
Boroecia alaska
Boroecia danae
Bellerophon erawanensis
Biarmeaspira mazaevi
Worthenia humiligrada
Heterosubulites longusapertura
Trypanocochlea lopburiensis
Euleptes europaea
Toxancyloceras canuti
Furcifer monoceras
Furcifer voeltzkowi
Carcinus maenas
Physa gyrina
Physa acuta
Physa fontinalis
Lebenharticrinus canaliculatus
Roveacrinus geinitzi
Styracocrinus peracutus
Roveacrinus alatus
Roveacrinus communis
Chalcides ocellatus
Testudo hermanni
Solitudo sicula
Garrigatitan meridionalis
Atsinganosaurus velauciensis
Aragosaurus ischiaticus
Pelusios castaneus
Pelusios carinatus
Pelusios rhodesianus
Pelusios nanus
Pelusios gabonensis
Pelusios chapini
Apistobranchus jasoni
Leitoscoloplos olei
Aphelochaeta falklandica
Dodecaceria saeria
Aphelochaeta longisetosa
Eurytemora affinis
Peripatus capensis
Peripatus leuckartii
Allodaposuchus hulki
Allodaposuchus precedens
Magelona equilamellae
Magelona 

Intralichen lichenicola
Epigloea urosperma
Abajudon kaayai
Proliferophorum thailandicum
Tamsiniella labiosa
Natrix astreptophora
Sindhochelys ragei
Sigilmassasaurus brevicollis
Spinosaurus maroccanus
Spinosaurus aegyptiacus
Microlicia seringeana
Microlicia martialis
Microlicia reichardtiana
Lavoisiera goyazensis
Campanula montana
Lepidochelys olivacea
Feralisaurus corami
Slooffia velesii
Hamamotoa cerberi
Hamamotoa telluris
Chernovia houtui
Ancyloceras patagonicum
Jenslehmannella bangestanense
Notosepia cliftonensis
Radix balthica
Lymnaea stagnalis
Metoposaurus krasiejowensis
Feeserpeton oklahomensis
Testudo hermanni
Tropidurus azurduyae
Tropidurus etheridgei
Eohyosaurus wolvaardti
Mesosuchus browni
Howesia browni
Hemihoplites feraudianus
Pancytospora epiphaga
Pedioplanis laticeps
Pedioplanis burchelli
Chelonoidis chilensis
Chalcides ocellatus
Chalcides parallelus
Fumagopsis stellae
Cryptosporidium testudinis
Natrix maura
Melanoides tuberculata
Preisingerella stegovnikensis
Lemurosauru

Avitograptus avitus
Metabolograptus persculptus
Avitograptus akidomorphus
Python regius
Laticauda colubrina
Notechis scutatus
Hydrelaps darwiniensis
Microcephalophis gracilis
Hydrophis stokesii
Hydrophis curtus
Aipysurus laevis
Microcephalophis melanocephalus
Hydrophis colubrina
Hydrophis peronii
Hydrophis cyanocinctus
Hydrophis atriceps
Hydrophis fasciatus
Hydrophis melanocephalus
Hydrophis parviceps
Hydrophis spiralis
Hydrophis caerulescens
Hydrophis ornatus
Hydrophis platurus
Hydrophis laevis
Laibalea enigmatica
Pristurus rupestris
Plasmodium malariae
Trichoplax adhaerens
Galba schirazensis
Podarcis hispanicus
Podarcis vaucheri
Chromatogenys tiliquoides
Nothosaurus mirabilis
Glomus tenue
Fortescuenella serenitatis
Baltograptus extremus
Pseudophyllograptus densus
Trigonotarbus johnsoni
Wronascolex antiquus
Wronascolex iacoborum
Palaeoscolex ratcliffei
Pleurosternon bullockii
Besanosaurus leptorhynchus
Actinoxylon banksii
Niassodon mfumukasi
Vectaerovenator inopinatus
Hatzegopteryx th

Platyceps collaris
Orobates pabsti
Gonyosoma prasinum
Gonyosoma coeruleum
Caretta caretta
Baryonyx walkeri
Proholopus holopiformis
Turrilites costatus
Turrilites acutus
Turrilites scheuchzerianus
Teratoscincus keyserlingii
Lepeophtheirus salmonis
Vipera walser
Apodomenia enigmatica
Intralichen lichenicola
Epigloea urosperma
Salvator merianae
Salvator rufescens
Tropidolaemus wagleri
Parias hageni
Daboia russelii
Coptospora paradoxa
Phimopollenites pannosus
Pilosisporites notensis
Pilosisporites parvispinosus
Slooffia velesii
Hamamotoa cerberi
Hamamotoa telluris
Chernovia houtui
Streblospio eridani
Tropidolaemus wagleri
Parias hageni
Daboia russelii
Sarcocystis frondea
Sarcocystis nipponi
Radix balthica
Ardoreosomus occidentalis
Candelarialepis argentus
Bystrowiella schumanni
Egertonodus basanus
Lonchidion breve
Lonchidion crenulatum
Vectiselachos ornatus
Hulkepholis rori
Turbanella hyalina
Radix lagotis
Protognathodus kockeli
Pelodiscus axenaria
Phelocyprideis acardomesido
Aleisocythere

Platymonas subcordiformis
Geochelone carbonaria
Sarcocystis frondea
Sarcocystis nipponi
Bryopa lata
Laternula elliptica
Laternula truncata
Lyonsia norwegica
Myadora brevis
Tropidomya abbreviata
Yelaphomte praderioi
Indotyphlops braminus
Afrotyphlops calabresii
Rhinotyphlops leucocephalus
Afrotyphlops cuneirostris
Afrotyphlops platyrhynchus
Euponera sikorae
Apicystis bombi
Rana cadurcorum
Thaumastosaurus gezei
Tynecotarbus tichaveki
Libonectes morgani
Chalcides viridanus
Anagaudryceras involvulum
Maledictosuchus riclaensis
Emys orbicularis
Natrix natrix
Hierophis viridiflavus
Stylolophus minor
Palaeoamasia kansui
Heviacrinus melendezi
Whatcheeria deltae
Lygodactylus madagascariensis
Intralichen lichenicola
Epigloea urosperma
Scolecolachnum nigricans
Fuscolachnum pteridis
Streblospio eridani
Nucras scalaris
Apostolepis tertulianobeui
Apostolepis assimilis
Pygsuia biforma
Tetracapsuloides bryosalmonae
Dicraeosaurus hansemanni
Polyplacotoma mediterranea
Rhodocodon petrae
Rhodocodon viridan

Banhxeochelys trani
Isometremys lacuna
Guangdongemys pingi
Macrostomum lignano
Baryonyx walkeri
Laternula elliptica
Lanthanotus borneensis
Roxochelys wanderleyi
Yuraramirim montealtensis
Varanus salvator
Pachybrachis sassii
Hemorrhois hippocrepis
Malpolon monspessulanus
Macroprotodon mauritanicus
Rhinechis scalaris
Ivoites opitzi
Natrix maura
Intralichen lichenicola
Epigloea urosperma
Podarcis hispanicus
Lacerta muralis
Podarcis guadarramae
Podarcis virescens
Pteraspis rostrata
Errivaspis waynensis
Taxomyces andreanae
Iberolacerta monticola
Typhloperipatus williamsoni
Cretoperipatus burmiticus
Furcifer monoceras
Furcifer voeltzkowi
Euperipatoides kanangrensis
Galba truncatula
Trachytherus ramirezi
Trachytherus alloxus
Trachytherus subandinus
Trachytherus spegazzinianus
Galba schirazensis
Varanus komodoensis
Dearc sgiathanach
Morellospora saccamoebae
Scolecolachnum nigricans
Fuscolachnum pteridis
Natrix helvetica
Natrix natrix
Solenopora jurassica
Neomelanconium gelatosporum
Neomelancon

In [77]:
backbone[backbone["canonicalName"]=="Galba truncatula"]

,canonicalName,order,family
4319213,Galba truncatula,NaN,Lymnaeidae


In [78]:
backbone_original[backbone_original["canonicalName"]=="Galba truncatula"]

,taxonID,datasetID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificName,scientificNameAuthorship,canonicalName,genericName,specificEpithet,...,namePublishedIn,taxonomicStatus,nomenclaturalStatus,taxonRemarks,kingdom,phylum,class,order,family,genus
5847115,10949887,7ddf754f-d193-4cc9-b351-99906754a03b,2291335.0,2291336.0,8246882.0,"Galba truncatula (O.F.Müller, 1774)","(O.F.Müller, 1774)",Galba truncatula,Galba,truncatula,...,NaN,synonym,NaN,NaN,Animalia,Mollusca,Gastropoda,NaN,Lymnaeidae,Galba
6554617,2291336,7ddf754f-d193-4cc9-b351-99906754a03b,2291335.0,NaN,8246882.0,"Galba truncatula (O.F.Müller, 1774)","(O.F.Müller, 1774)",Galba truncatula,Galba,truncatula,...,NaN,accepted,NaN,NaN,Animalia,Mollusca,Gastropoda,NaN,Lymnaeidae,Galba


In [ ]:
                    

# DISAMBIGUATE
def match(a, b):
    same = False
    # if no known orders for one of them, institution and full name must match 
    if a.order == [] or b.order == []:
        if a.inst_id == b.inst_id and a.author_display_name==b.author_display_name:
            same = True
    # if both have known orders, orders and institution must match
    else:
        if a.inst_id == b.inst_id and len(set(a.order).intersection(set(b.order))) > 0:
            same = True
    
    return same


def cluster(matches):
    clusters = []
    
    for match in matches:
        # check if it matches any existing groups
        match_with_groups = []
        for i, group in enumerate(clusters):
            if len(set(match).intersection(set(group))) > 0:
                match_with_groups.append(i)
        
        # if it fits with no existing group, add it by itself
        if len(match_with_groups) == 0:
            clusters.append(match)
        # if it fits with one existing group, add it to that group
        elif len(match_with_groups) == 1:
            clusters[match_with_groups[0]].extend(match)
        # if it fits with multiple groups, mash those groups together
        else:
            print(clusters) # apparently this never happens
            supergroup = []
            # remove each group and add its contents to the new supergroup
            for j in match_with_groups.sort(reverse=True):
                supergroup.extend(clusters.pop(j))
            supergroup.extend(match)
            clusters.append(supergroup)
            print(clusters)
    
    # turn into a list of sets to get unique values
    clusters = [set(x) for x in clusters] 
    return clusters


# emergency meeting: go over every duplicated name
true_people = []
duplicates = authors[authors.duplicated(subset=["truncatedName"], keep=False)]

for name in set(duplicates["truncatedName"]):
    # get all trund names that are exact string matches to this name
    #same_names = duplicates[duplicates["author_display_name"]==name]
    same_names = duplicates[duplicates["truncatedName"]==name]
    matches = []
    
    for person_a in same_names.itertuples():
        aliases = [person_a.Index,]
        
        for person_b in same_names.itertuples():
            if match(person_a, person_b) and person_a.Index!=person_b.Index:
                aliases.append(person_b.Index)
                
        matches.append(aliases)

    true_people.extend(cluster(matches))


true_authors = authors[-(authors.duplicated(subset="truncatedName", keep=False))].reset_index()

merged_people = []
m = 1

def collect_values(df, person_ids, column):
    if len(person_ids) > 0:
        values = []
        
        for duplicate in person_ids:
            imposter = df.loc[duplicate]
            if imposter[column] not in values and imposter[column] != None:
                if column == "order" or column == "species_subject":
                    values.extend(imposter[column])
                else:
                    values.append(imposter[column])
                
    else:
        values = df.loc[person_ids, column]

    values = set(values)
    if len(values)==1:
        values = list(values)[0]
        
    return values

person_ids = []
m = 1

for person in true_people:
    row = [person,]
    for column in true_authors.columns[1:]:
        answer = collect_values(authors, person, column)
        row.append(answer)
        
    merged_people.append(authors.loc[list(person)].reset_index())
    merged_people[-1]["groupNr"] = m
    m += 1
    
    true_authors.loc[len(true_authors)] = row
    
    
merged_df = pd.concat(merged_people, ignore_index=True)        
merged_df.to_csv("../../data/interim/merged_people_truncated.csv")

true_authors.to_pickle("../../data/processed/european_authors_disambiguated_truncated.pkl")
true_authors.to_csv("../../data/processed/european_authors_disambiguated_truncated.tsv", sep="\t")
